## Extracting Transforming Load (ETL)

En este notebook se planea procesar los datos encontrados en los archivos .csv. Con este fin vamos a usar distintas librerias que nos permitan obtener una base de datos limpia. Es de tener en cuenta que se usaran la menor cantidad de celdas de accion con el fin de hacer nuestro codigo lo mas corto y simple posible. De ser necesario, las interacciones tendran su respectivo comentario que nos ayudara a entender el codigo.

Nuestro codigo se repartira en distintas secciones teniendo una similitud con el patron Modelo - Vista - Controlador (MVC):

Celda de librerias.
Celda de acciones.
Celda de vistas.


## 1. Librerias
Esta Celda nos ayudara a traer todas las librerias necesarias con el fin de completar el ejercicio

In [4]:
#Importamos las librerias necesarias para el ejercicio

import pandas as pd
import numpy as np

## 2. Acciones
Esta celda nos ayudara a crear todas las funciones necesarias

In [3]:
#Funcion que crea diccionarios para poder contar los tipos de datos que se encuentran en cada dataframe
def diccionario_tipos_datos(dataframe_file):

    dicc_col = {}
    
    for columna in dataframe_file:
        conteo_tipos = dataframe_file[columna].apply(type).value_counts().to_dict()
        dicc_col[columna] = {str(tipo): conteo for tipo, conteo in conteo_tipos.items()}

    return dicc_col

#Funcion que convierte los datos a flotantes si son numeros o ceros si son de otro tipo

def cambio_a_float(dato):

    if pd.isna(dato):
        return 0.0
    
    try:
        return float(dato)
    except(ValueError, TypeError):
        return 0.0

#Funcion que convierte los datos a Datetime y devuelve None en caso de ser otro tipo

def cambio_a_datetime(dato):

    if pd.isna(dato):
        return None

    try:
        return pd.to_datetime(dato)
    except ValueError:
        return None

## 3. Vistas
## 3.1. Obtencion de datos

En esta seccion nos enfocamos en transformar los datos del formato original (.csv) a un formato Dataframe de pandas con el fin de poder observar, eliminar, transformar y demas procesos que nos permitan una mejor visualizacion y optimizacion de dichos datos.

Los procesos a aplicar son:

* Transformacion de .csv a Dataframe

* Eliminacion y agregacion de columnas

* Eliminacion de valores Null/Nan en los distintos Datasets

* Reinicio de indices

* Casting de datos a formatos correspondientes



## 3.1.1 ETL para el Dataset: "Internet_Penetracion.csv"

In [19]:
#Carga del dataset
inter_hogar = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Penetracion.csv", sep=",")
print("Carga completa del dataset: Internet_Penetracion.csv")

#Remplazo de "," a "." en columnas: Accesos por cada 100 hogares; Accesos por cada 100 hab
col_trabajar = ["Accesos por cada 100 hogares","Accesos por cada 100 hab"]
inter_hogar["Accesos por cada 100 hogares"] = inter_hogar["Accesos por cada 100 hogares"].str.replace(",", ".")
inter_hogar["Accesos por cada 100 hab"] = inter_hogar["Accesos por cada 100 hab"].str.replace(",", ".")
print(f"Caracteres cambiados en columnas: {col_trabajar}")

#Casting str a float
inter_hogar["Accesos por cada 100 hogares"] = inter_hogar["Accesos por cada 100 hogares"].apply(cambio_a_float)
inter_hogar["Accesos por cada 100 hab"] = inter_hogar["Accesos por cada 100 hab"].apply(cambio_a_float)
print(f"Cambio format a float en columnas: {col_trabajar}")

Carga completa del dataset: Internet_Penetracion.csv
Caracteres cambiados en columnas: ['Accesos por cada 100 hogares', 'Accesos por cada 100 hab']
Cambio format a float en columnas: ['Accesos por cada 100 hogares', 'Accesos por cada 100 hab']


## 3.1.2 ETL para el Dataset: "Internet_BAF.csv"

In [27]:
#Carga del dataset
inter_tech = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_BAF.csv",sep=",")

#Eliminacion de la col "total" y adicion de una nueva
inter_tech.drop(columns="Total", inplace=True)
inter_tech["total"] = inter_tech["Banda ancha fija"] + inter_tech["Dial up"]
print("Columna: Total modificada")


Columna: Total modificada


In [28]:
inter_tech.head()

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,total
0,2022,4,Buenos Aires,5,6.0,11.0
1,2022,4,Capital Federal,2,2.0,4.0
2,2022,4,Catamarca,71,1.0,72.0
3,2022,4,Chaco,144,5.0,149.0
4,2022,4,Chubut,168,904.0,1072.0


In [25]:
inter_tech.isnull().sum()

Año                 0
Trimestre           0
Provincia           0
Banda ancha fija    0
Dial up             2
Total               0
dtype: int64

In [26]:
diccionario_tipos_datos(inter_tech)

{'Año': {"<class 'int'>": 864},
 'Trimestre': {"<class 'int'>": 864},
 'Provincia': {"<class 'str'>": 864},
 'Banda ancha fija': {"<class 'int'>": 864},
 'Dial up': {"<class 'float'>": 864},
 'Total': {"<class 'int'>": 864}}

In [ ]:
#Carga del dataset

inter_acess = pd.read_csv("C:\\Users\\andre\\Documents\\Py Codes\\data\\Internet_Accesos-por-conectividad.csv",sep=",")
